### 0. import the required packages to read the files

In [ ]:
# import packages as you need them here


### 1. Ingesting raw data files

In [ ]:
path = 'your file path'

raw_file = 'Lekagul Sensor Data.csv'

loc_file2 = 'coordinates_small.csv' # self created file using image provided with dataset

In [ ]:
# read in the file
raw_data = 

In [ ]:
# look at the first 10 elements


In [ ]:
# examine the data types of each column


In [ ]:
# shape of the df


In [ ]:
# print out the different gate name


In [ ]:
# count the number of unique gate names


### 2. Need to convert dtype for time field
- change the date data type  using **pd.to_datetime(field, format='%d/%m/%Y %H:%M')**

In [ ]:
raw_data['Timestamp'] = 

In [ ]:
# check data types again to make sure


### 3. De-duplication before any analysis
- think of a method to find all the duplicated rows in the data
- find the total number of duplicated rows

In [ ]:
# create a bool column for the duplicated records
raw_data['is_duplicated'] = 

In [ ]:
# get the sum to get a total count of duplicate records


#### 3.1 separate duplicated records

In [ ]:
# collect the duplicated records
duplicatedrecords = 

#### 3.2 Remove duplicate records

In [ ]:
raw_data = 

In [ ]:
raw_data.head()

### 4. Create ExitTime and ExitGate 

In [ ]:
# rename exisiting fields for convenience
raw_data = raw_data.rename(columns={'Timestamp': 'EntryTime', 'gate-name': 'EntryGate'})
raw_data.head()

In [ ]:
# create new columns using shift() method with grouby on car-id
raw_data['ExitTime'] = raw_data.groupby('car-id')['EntryTime'].shift(1)

In [ ]:
# use the same logic as above
raw_data['ExitGate'] = 

In [ ]:
raw_data.head()

### 4.1 Remove NaN or NaT introduced due to shift()

In [ ]:
raw_data = 

In [ ]:
raw_data.head()

### 4.2 Remove the is_duplicated as it has served its purpose
- del function

In [ ]:
# delete the column

In [ ]:
# verify using shape

### 5. Create column for time spent at between 2 consecutive locations
### This will be used to compute speed (note: speedlimit 25 mph)

In [ ]:
#above checks looks fine so let's create time spent field
raw_data['SecondSpent'] = (raw_data['EntryTime'] - raw_data['ExitTime']).dt.total_seconds()
raw_data['MinuteSpent'] =    # explore as shown above with dot (.) operator
raw_data['DaysSpent'] =      # explore as shown above with dot (.) operator

### 6. Date field to generated to create a filter for visualization
- **just run it to create this feature**

In [ ]:
raw_data['Date'] =  raw_data.EntryTime.dt.date 

### 7. Remove erreneous records:
- where timestamp is 1970-01-01 00:00:00 (introduced in cell above)
- where timestamps between 2 locs is same but the gates are different suggesting an error in data entry

In [ ]:
# where timestamp is 1970-01-01 00:00:00 set the time related fields to 0


#### 7.1 Flag errors

In [ ]:
# create a bool
# where timestamps between 2 locs is same but the gates are different suggesting an error in data entry
raw_data['error'] = 

In [ ]:
raw_data['error'].value_counts()

In [ ]:
# remove error == True and save it to raw_data again
raw_data = 

In [ ]:
# delete the error column


#### Let's rearrange the columns to place similar ones together

In [ ]:
columns = raw_data.columns.tolist()
columns

In [ ]:
# create a list
new_columns = ['car-id','car-type','Date','EntryTime','ExitTime','EntryGate', 'ExitGate','SecondSpent',
 'MinuteSpent', 'DaysSpent']

In [ ]:
raw_data = raw_data[new_columns]

In [ ]:
print raw_data.shape
raw_data.head()

### 8. Let's bring in location coordinates for EntryGate and ExitGate
#### 8.1 First replace the 0s in exitgates with entry gates so that distance and speed computations can be carried out

In [ ]:
raw_data.ExitGate[raw_data.ExitGate==0] = raw_data.EntryGate

In [ ]:
raw_data.head()

#### 8.2 Read in the coordinates from the loc_file2 and get the X and Y coordinates

In [ ]:
loc_data = pd.read_csv(os.path.join(path, loc_file2))

In [ ]:
loc_data['coordinates'] = zip(loc_data.X, loc_data.Y)
#loc_data['coordinates']

In [ ]:
# write a utility function for lookup
def FindCoordinates(gatename):
    coo = loc_data.loc[loc_data['gate-name'] == gatename, 'coordinates'].iloc[0]
    return coo

In [ ]:
# write a utility function for lookup
def FindCoordinatesX(gatename):
    coo = loc_data.loc[loc_data['gate-name'] == gatename, 'coordinates'].iloc[0]
    return coo[0]

In [ ]:
# write a utility function for lookup
def FindCoordinatesY(gatename):
    coo = loc_data.loc[loc_data['gate-name'] == gatename, 'coordinates'].iloc[0]
    return coo[1]

In [ ]:
# test a function
print FindCoordinates('ranger-stop2')
print FindCoordinatesX('ranger-stop2')
print FindCoordinatesY('ranger-stop2')

In [ ]:
raw_data['EntryX']=raw_data['EntryGate'].apply(lambda cell :FindCoordinatesX(cell))
raw_data['EntryY'] = raw_data['EntryGate'].apply(lambda cell :FindCoordinatesY(cell))


In [ ]:
raw_data['ExitX']=raw_data['ExitGate'].apply(lambda cell :FindCoordinatesX(cell))
raw_data['ExitY'] = raw_data['ExitGate'].apply(lambda cell :FindCoordinatesY(cell))

In [ ]:
raw_data['ExitCoordinates'] = raw_data['ExitGate'].apply(lambda row: FindCoordinates(row))

In [ ]:
raw_data['EntryCoordinates'] = raw_data['EntryGate'].apply(lambda row: FindCoordinates(row))

In [ ]:
print raw_data.shape
raw_data.head()

### 9. Let's find out the distance moved (straight-line) and derive speed of movement
#### 982 units = 12 mile for the image used<br/>
#### 1 unit = 12/982 miles

In [ ]:
from scipy.spatial import distance
def DistanceMoved(df):    
    miles = round(distance.euclidean(df['EntryCoordinates'],df['ExitCoordinates'])*(12.0/982.0),3)
    return miles 

In [ ]:
# test
df = pd.DataFrame({'ID':['1','2'], 'EntryCoordinates': [(564, 818),(394, 174)], 'ExitCoordinates':[(316, 128),(440, 82)]})
df.apply(DistanceMoved, axis=1)

In [ ]:
# apply the given function on the dataframe
raw_data['DistanceMiles'] = 

### 10. Similarily compute speed of movement between coordinates

In [ ]:
def Speed(df):
    distance = df['DistanceMiles']
    hours = df['SecondSpent']/3600.0
    if hours > 0.0:
        return round(distance/hours,3)
    else:
        return 0

In [ ]:
# apply the Speed function
raw_data['SpeedMPH'] = 

In [ ]:
raw_data['SpeedMPH'].describe()

In [ ]:
raw_data['NoMovement']=(raw_data['SpeedMPH']==0)

In [ ]:
# instances where vehicles exceeded speed limit of 25
raw_data['SpeedLimitViolation']=

#### Let's keep only required columns

In [ ]:
# let's keep only required columns
# change is as per need
new_columns = ['car-id',
 'car-type',
 'Date',
 'EntryTime', 
 'EntryGate',
 'SecondSpent',
 'MinuteSpent',
 'DaysSpent',
 'EntryX',
 'EntryY',
 'EntryCoordinates',
 'DistanceMiles',
 'SpeedMPH',
 'NoMovement',
 'SpeedLimitViolation']

In [ ]:
raw_data = raw_data[new_columns]

#### save the work to a cleaned file

In [ ]:
file_name = 'cleandata_activity.csv'

In [ ]:
raw_data.to_csv(os.path.join(path, file_name), sep=';', encoding='utf-8', index=False)

### 11. Read back the clean dataset

In [ ]:
file_name = 'cleandata_activity.csv'

In [ ]:
# read the file back to make sure
clean_data = pd.read_csv(os.path.join(path, file_name), delimiter=';')

In [ ]:
print clean_data.shape
clean_data.head()

In [ ]:
#clean_data.to_csv(os.path.join(path, 'full2.csv'), sep=',', encoding='utf-8', index=False)